# Tutorial PySqpark e MLlib

### Instalação e configuração das bibliotecas e suas dependências

In [1]:
# Baixando e instalando o Java (JDK)
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #Java

# Baixando e instalando o ecossistema Hadoop com o Spark
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7-hive1.2.tgz
!tar xf spark-3.0.1-bin-hadoop2.7-hive1.2.tgz

# Importando as bibliotecas Python para executar o Spark
!pip install -q findspark
!pip install -q pyspark

### Inicialização do Spark

In [2]:
import findspark

findspark.init("/content/spark-3.0.1-bin-hadoop2.7-hive1.2")

### Inicialização de uma Sessão no Spark

In [3]:
# Importando o módulo de sessões do Spark
from pyspark.sql import SparkSession

# Criando uma sessão do Spark
spark = SparkSession.builder \
    .master("local") \
    .appName("Linear Regression Model") \
    .config("spark.executor.memory", "1gb") \
    .getOrCreate()

# Armazenando o endereço do programa SparkContext
sc = spark.sparkContext

### Importação de um arquivo no Google Colab

In [4]:
# Importando o módulo de manipulação de arquivos do Google Colab
from google.colab import files

# Fazendo o upload de um arquivo
uploaded = files.upload()

Saving Salary_Data.csv to Salary_Data (2).csv


### Manipulação de RDD's

In [5]:
# Lendo um conjunto de dados num RDD
rdd = sc.textFile('Salary_Data.csv')

In [6]:
# Apresentando os primeiros 2 registros do conjunto de dados 
rdd.take(2)

['1.1,39343.00', '1.3,46205.00']

In [7]:
# Sperando os valores por um delimitador (vírgula)
rdd = rdd.map(lambda line: line.split(","))

# Apresentando o primeiro registro do conjunto de dados pós separado
rdd.take(1)

[['1.1', '39343.00']]

In [8]:
# Apresentando o primeiro registro 
rdd.first()

['1.1', '39343.00']

In [9]:
# Apresentando o(s) primeiro(s) n registros do conjunto de dados
rdd.top(1)

[['9.6', '112635.00']]

In [10]:
# Importando o modulo de registros do Spark
from pyspark.sql import Row

# Criando um dataframe com o RDD separado em anos de experiência (posição 0) e salário (posição 1)
df = rdd.map(lambda line: Row(YearsExperience=line[0], Salary=line[1])).toDF()

In [11]:
# Apresentando os primeiros 20 (padrão) registros 
df.show()

+---------------+--------+
|YearsExperience|  Salary|
+---------------+--------+
|            1.1|39343.00|
|            1.3|46205.00|
|            1.5|37731.00|
|            2.0|43525.00|
|            2.2|39891.00|
|            2.9|56642.00|
|            3.0|60150.00|
|            3.2|54445.00|
|            3.2|64445.00|
|            3.7|57189.00|
|            3.9|63218.00|
|            4.0|55794.00|
|            4.0|56957.00|
|            4.1|57081.00|
|            4.5|61111.00|
|            4.9|67938.00|
|            5.1|66029.00|
|            5.3|83088.00|
|            5.9|81363.00|
|            6.0|93940.00|
+---------------+--------+
only showing top 20 rows



In [12]:
# Apresentando o schema (tipos de dados) do conjunto de dados
df.printSchema()

root
 |-- YearsExperience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [13]:
# Importando os módulos de tipos do Spark
from pyspark.sql.types import *

# Função que converte o(s) tipo(s) de coluna(s) num dataframe do Spark
def convertColumn(df, names, newType):
    for name in names: 
        df = df.withColumn(name, df[name].cast(newType))
    return df 

# Definindo a lista de colunas a ser convertidas
columns = ['YearsExperience', 'Salary']

# Convertendo as colunas para tipo float
df = convertColumn(df, columns, FloatType())

In [14]:
# Apresentando novamente o schema com os tipos alterados
df.printSchema()

root
 |-- YearsExperience: float (nullable = true)
 |-- Salary: float (nullable = true)



In [15]:
# Apresentando os 20 primeiros registros do dataframe 
df.show()

+---------------+-------+
|YearsExperience| Salary|
+---------------+-------+
|            1.1|39343.0|
|            1.3|46205.0|
|            1.5|37731.0|
|            2.0|43525.0|
|            2.2|39891.0|
|            2.9|56642.0|
|            3.0|60150.0|
|            3.2|54445.0|
|            3.2|64445.0|
|            3.7|57189.0|
|            3.9|63218.0|
|            4.0|55794.0|
|            4.0|56957.0|
|            4.1|57081.0|
|            4.5|61111.0|
|            4.9|67938.0|
|            5.1|66029.0|
|            5.3|83088.0|
|            5.9|81363.0|
|            6.0|93940.0|
+---------------+-------+
only showing top 20 rows



In [16]:
# Apresentando os salários dos primeiros 10 registros
df.select('Salary') \
    .show(10)

+-------+
| Salary|
+-------+
|39343.0|
|46205.0|
|37731.0|
|43525.0|
|39891.0|
|56642.0|
|60150.0|
|54445.0|
|64445.0|
|57189.0|
+-------+
only showing top 10 rows



In [17]:
# Agrupando o dataframe por salários e contabilizando os diferentes valores
df.groupBy("Salary") \
    .count() \
    .sort("Salary", ascending=False) \
    .show()

+--------+-----+
|  Salary|count|
+--------+-----+
|122391.0|    1|
|121872.0|    1|
|116969.0|    1|
|113812.0|    1|
|112635.0|    1|
|109431.0|    1|
|105582.0|    1|
|101302.0|    1|
| 98273.0|    1|
| 93940.0|    1|
| 91738.0|    1|
| 83088.0|    1|
| 81363.0|    1|
| 67938.0|    1|
| 66029.0|    1|
| 64445.0|    1|
| 63218.0|    1|
| 61111.0|    1|
| 60150.0|    1|
| 57189.0|    1|
+--------+-----+
only showing top 20 rows



In [18]:
# Apresentando as estatísticas do dataframe (quantidade, média, desvio padrão, mínimo e máximo)
df.describe().show()

+-------+------------------+------------------+
|summary|   YearsExperience|            Salary|
+-------+------------------+------------------+
|  count|                30|                30|
|   mean|5.3133333643277485|           76003.0|
| stddev|2.8378881722287805|27414.429784582302|
|    min|               1.1|           37731.0|
|    max|              10.5|          122391.0|
+-------+------------------+------------------+



### Preparação dos Dados

In [19]:
# Importando o módulo de tipo vetor denso para preparação dos dados para aplicar a regressão linear
from pyspark.ml.linalg import DenseVector

# Criando um novo dataframe no formato [(salário, DenseVector(anos de experiência))]
input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Substituindo o dataframe antigo pelo novo criado anteriormente
df = spark.createDataFrame(input_data, ["label", "features"])

In [20]:
# Importando o módulo de normalização de dados 
from pyspark.ml.feature import StandardScaler

# Inicializando a normalização dos dados com as configurações necessárias (coluna(s) de entrada e saída)
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Aplicando a normalização dos dados
scaler = standardScaler.fit(df)

# Transformando o dataframe com os dados normalizados
scaled_df = scaler.transform(df)

# Apresentando os primeiros 2 registros do dataframe
scaled_df.take(2)

[Row(label=1.100000023841858, features=DenseVector([39343.0]), features_scaled=DenseVector([1.4351])),
 Row(label=1.2999999523162842, features=DenseVector([46205.0]), features_scaled=DenseVector([1.6854]))]

In [21]:
# Separando os dados em treino (75%) e teste (25%)
train_data, test_data = scaled_df.randomSplit([.75,.25], seed=1234)

### Processamento dos Dados

Crianção do(s) Modelo(s)

In [22]:
# Importando o módulo de regressão linear do Spark
from pyspark.ml.regression import LinearRegression

# Inicializando a regressão linear com as configurações necessárias (coluna com os rótulos e número máximo de iterações)
lr = LinearRegression(labelCol="label", maxIter=10)

# Aplicando a regressão linear nos dados de treino e criando o modelo
linearModel = lr.fit(train_data)

In [23]:
# Gerando as predições com modelo sobre os dados de teste
predicted = linearModel.transform(test_data)

# Recuperando as predições e rótulos dos dados de teste
predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])

# Combinando as predições com os rótulos dos dados de teste
predictionAndLabel = predictions.zip(labels).collect()

# Apresentando os primeiros 5 registros com as predições e rótulos dos dados de teste 
predictionAndLabel[:5]

[(2.4088336346189774, 1.2999999523162842),
 (1.8115026389699527, 2.200000047683716),
 (7.334685753949579, 7.099999904632568),
 (8.390279553355947, 8.699999809265137),
 (8.026147940708958, 9.0)]

In [24]:
# Apresentando o(s) coeficiente(s) angulare(s) do modelo (beta1)
linearModel.coefficients

DenseVector([0.0001])

In [25]:
# Apresentando o intercepto do modelo (beta0)
linearModel.intercept

-1.962353988751815

Avaliação de Métricas

In [26]:
# Apresentando o erro quadrático médio do modelo (RMSE)
linearModel.summary.rootMeanSquaredError

0.5604487702361474

In [27]:
# Apresentando o R2 do modelo
linearModel.summary.r2

0.9347632503845014

### Finalização da Sessão do Spark

In [28]:
# Encerrando a sessão do Spark
spark.stop()